# Bali International Tourist Visit Data Compiling

The BPS, Indonesia's Statistic Center Body, is a good source for information. However, their datasets are formatted for display, not analysis. We will trim the datasets and compile them into a complete dataset, consisting of month, year, visitor through airport, visitor through seaport, and total visitor. This allow use for analysis.

In [13]:
import numpy as np
import pandas as pd

Take a look at a sample dataset

In [14]:
df = pd.read_csv('./Raw Datasets/Banyaknya Wisatawan Mancanegara Bulanan ke Bali Menurut Pintu Masuk, 2025.csv')
df

,Pintu Masuk Wisman,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,Banyaknya Wisatawan Mancanegara Bulanan ke Bal...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Januari,Februari,Maret,April,Mei,Juni,Juli,Agustus,September,Oktober,November,Desember,Tahunan
3,Bandara Ngurah Rai,526831,444411,464697,587315,602055,637444,697068,-,-,-,-,-,-
4,Pelabuhan Laut,3066,6286,6154,3906,158,124,39,-,-,-,-,-,-
5,Jumlah,529897,450697,470851,591221,602213,637868,697107,-,-,-,-,-,-


Problems of this Datasets:
- There are empty spaces. Row 0 and 1 only contain the document's title and year.
- Row 2, despite containing month of the year, is unusable because it's in Bahasa.
- Columns are unnamed

Solution:
- Drop Row 0 to 2
- Change column names to 0-13

In [15]:
df = df.drop([0,1,2])

col_index = []
for i in range(14): col_index.append(i)
    
df.columns = col_index
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
3,Bandara Ngurah Rai,526831,444411,464697,587315,602055,637444,697068,-,-,-,-,-,-
4,Pelabuhan Laut,3066,6286,6154,3906,158,124,39,-,-,-,-,-,-
5,Jumlah,529897,450697,470851,591221,602213,637868,697107,-,-,-,-,-,-


The dataset looks better, but still need further transformation: 
- Drop column 13. This column contains total yearly visitor.
- Set index to column 0. When we transpose the dataset, this column will become the header row.
- Transpose dataset.
- Reset index. We will use the index as month.
- Create column "Year" and assign the dataset year to it.
- Change column names (optional)
- Create column "Period" from combining month and year, set as index
- Rearrange the columns to: Month, Year, Ngurah Rai Airport, Seaport, Total

In [16]:
df = df.drop(columns=[13]) # Drop total visitor column
df = df.set_index(0) # Set column 0 as index
df = df.transpose() # Transpose dataset
df = df.reset_index() # Reset index
df["Year"] = 2025 # Set year
df = df.rename(columns={"index":"Month", "Bandara Ngurah Rai":"Ngurah Rai Airport", "Pelabuhan Laut":"Seaport", "Jumlah":"Total"}) # Rename columns
df["Period"] = pd.to_datetime(dict(year=df["Year"], month=df["Month"], day=1)) # Create column "Period" by combining year and month and set as datetime type
df["Period"] = df["Period"].dt.to_period("M") # Change column "Period" format to monthly. (Default is day-month-year)
df = df.set_index("Period") # Set "Period" as index
df = df.iloc[:, [0, 4, 1, 2, 3]] # Rearrange columns
df

,Month,Year,Ngurah Rai Airport,Seaport,Total
Period,,,,,
2025-01,1,2025,526831,3066,529897
2025-02,2,2025,444411,6286,450697
2025-03,3,2025,464697,6154,470851
2025-04,4,2025,587315,3906,591221
2025-05,5,2025,602055,158,602213
2025-06,6,2025,637444,124,637868
2025-07,7,2025,697068,39,697107
2025-08,8,2025,-,-,-
2025-09,9,2025,-,-,-


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 12 entries, 2025-01 to 2025-12
Freq: M
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Month               12 non-null     int64 
 1   Year                12 non-null     int64 
 2   Ngurah Rai Airport  12 non-null     object
 3   Seaport             12 non-null     object
 4   Total               12 non-null     object
dtypes: int64(2), object(3)
memory usage: 576.0+ bytes


Last, change the "-" to 0 and change the non numerical data type to int64

In [18]:
df = df.replace("-",0) # Replace "-"

# Change data type
for i in range(2,5):
    df[df.columns[i]] = df[df.columns[i]].astype(int)
df

,Month,Year,Ngurah Rai Airport,Seaport,Total
Period,,,,,
2025-01,1,2025,526831,3066,529897
2025-02,2,2025,444411,6286,450697
2025-03,3,2025,464697,6154,470851
2025-04,4,2025,587315,3906,591221
2025-05,5,2025,602055,158,602213
2025-06,6,2025,637444,124,637868
2025-07,7,2025,697068,39,697107
2025-08,8,2025,0,0,0
2025-09,9,2025,0,0,0


## Applying preprocessing steps and Compiling the datasets

We will apply the preprocessing steps to the datasets and combine them. First, create a function based on previous steps to make it replicatable.

In [19]:
def preprocessing(df, year_num):
    df = df.drop([0,1,2]) # Drop row 0-2. We can't use
    df.columns = col_index
    df = df.drop(columns=[13])
    df = df.set_index(0)
    df = df.transpose()
    df = df.reset_index()
    df["Year"] = year_num
    df = df.rename(columns={"index":"Month", "Bandara Ngurah Rai":"Ngurah Rai Airport", "Pelabuhan Laut":"Seaport", "Jumlah":"Total"})
    df["Period"] = pd.to_datetime(dict(year=df["Year"], month=df["Month"], day=1))
    df["Period"] = df["Period"].dt.to_period("M")
    df = df.set_index("Period")
    df = df.iloc[:, [0, 4, 1, 2, 3]]
    df = df.replace("-",0)    
    for i in range(2,5):
        df[df.columns[i]] = df[df.columns[i]].astype(int)
    return df

Then, create a function to compile the datasets

In [20]:
def compile_dataset(path_prefix, start_year, end_year):
    path = f"{path_prefix}{start_year}.csv"
    df1 = pd.read_csv(path)
    df1 = preprocessing(df1, start_year)

    for i in range(start_year+1, end_year+1):
        path = f"{path_prefix}{i}.csv"
        df2 = pd.read_csv(path)
        df2 = preprocessing(df2, i)
        df1 = pd.concat([df1, df2])

    return(df1)

Last, define the parameters and call the function

In [21]:
prefix = "./Raw Datasets/Banyaknya Wisatawan Mancanegara Bulanan ke Bali Menurut Pintu Masuk, "
start_year = 2009
end_year = 2025

col_index = []
for i in range(14): col_index.append(i)

df = compile_dataset(prefix, start_year, end_year)

In [22]:
df

,Month,Year,Ngurah Rai Airport,Seaport,Total
Period,,,,,
2009-01,1,2009,173867,674,174541
2009-02,2,2009,146115,1589,147704
2009-03,3,2009,167954,251,168205
2009-04,4,2009,188189,587,188776
2009-05,5,2009,190638,165,190803
...,...,...,...,...,...
2025-08,8,2025,0,0,0
2025-09,9,2025,0,0,0
2025-10,10,2025,0,0,0


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 204 entries, 2009-01 to 2025-12
Freq: M
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Month               204 non-null    int64
 1   Year                204 non-null    int64
 2   Ngurah Rai Airport  204 non-null    int64
 3   Seaport             204 non-null    int64
 4   Total               204 non-null    int64
dtypes: int64(5)
memory usage: 9.6 KB


The combined dataset is ready to export

In [24]:
prefix = "./Compiled Dataset/Banyaknya Wisatawan Mancanegara Bulanan ke Bali Menurut Pintu Masuk, "
df.to_csv(f"{prefix}{start_year}-{end_year}.csv")